In [27]:
import pandas as pd

# use a function to support finding the same car but different names in two above websites
from difflib import get_close_matches

def find_closest_match(target, alist):
    # Use difflib's get_close_matches to find the closest match
    closest_matches = get_close_matches(target, alist, n=1, cutoff=0.0)
    return closest_matches[0] if closest_matches else None

In [2]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import re

def get_html(url, host):
    ua = UserAgent()
    headers = {'User-Agent': ua.random,
            #    'Cookie': cookies,
               'Host':host}
    req = Request(url, headers=headers)
    html = urlopen(req).read().decode('utf-8')
    return html

In [3]:
kudo_url = 'https://www.kudosprime.com/fh5/carlist.php?range=2000' # list all cars from kudosprime. Note: this website has lots of errors in full names (see below)
kudo_html = get_html(kudo_url, host='www.kudosprime.com')
kudo_soup = BeautifulSoup(kudo_html)

In [4]:
all_car_info = kudo_soup.find_all('div', {'id':"carlist"})[0] # grab info e.g., Make
groupby_car_info = all_car_info.find_all('p', {'class':"groupby"})
num_list, make_list = [], []
for each in groupby_car_info:
    num, make = int(re.findall('\?/(\d+)', each.get_text())[0]), each.find_all('a')[0].get_text()
    num_list.append(num)
    make_list.append(make.upper())

In [5]:
sum(num_list)

893

In [6]:
all_car_info = kudo_soup.find_all('div', {'id':"carlist"})[0]
all_car_info = all_car_info.find_all('div', {'class':re.compile('car .*?')})
len(all_car_info)

893

In [7]:
short_name_lst, full_name_lst = [], [] # grab each car info e.g., full name
for each in all_car_info:
    short_name = each.find('span', class_='shortname')
    short_name = short_name.text if short_name else "Not Found"
    full_name = each.find_all('a', {'class': 'name'})[0].contents[0].strip()
    full_name = full_name.replace('"', "'").upper()
    short_name_lst.append(short_name.upper())
    full_name_lst.append(full_name.upper())
len(short_name_lst), full_name_lst[:5]

(893,
 ['1968 ABARTH 595 ESSEESSE',
  '1980 ABARTH FIAT 131',
  '2016 ABARTH 695 BIPOSTO',
  '2017 ABARTH 124 SPIDER',
  '2001 ACURA INTEGRA TYPE-R'])

In [8]:
i = 0
car_make_list = []
for each in num_list:
    for _ in range(each):
        car_make_list.append(make_list[i])
    i += 1
len(car_make_list)

893

In [9]:
# Merge car make, car model name here
df_kudo = pd.DataFrame({'CAR MAKE':car_make_list,
                        'CAR MODEL(Full Name)':full_name_lst,
                        'CAR MODEL(Short Name)':short_name_lst})
df_kudo = df_kudo[df_kudo['CAR MODEL(Full Name)'] != '2021 MERCEDES-AMG MERCEDES-AMG ONE'] # this one doesn't exist, maybe available for future.
df_kudo.head()

,CAR MAKE,CAR MODEL(Full Name),CAR MODEL(Short Name)
0,ABARTH,1968 ABARTH 595 ESSEESSE,ABARTH 595 '68
1,ABARTH,1980 ABARTH FIAT 131,ABARTH 131
2,ABARTH,2016 ABARTH 695 BIPOSTO,ABARTH 695 '16
3,ABARTH,2017 ABARTH 124 SPIDER,ABARTH 124 '17
4,ACURA,2001 ACURA INTEGRA TYPE-R,ACURA INTEGRA


In [10]:
df_kudo.loc[df_kudo['CAR MODEL(Full Name)'] == '2011 AUDI RS 5 COUPÉ', 'CAR MODEL(Full Name)'] = "2011 AUDI RS 5 COUPE" # modify this first, because our diff function fails to fix this

In [11]:
df_kudo.shape

(892, 3)

In [12]:
fh5_url = 'https://forza.net/fh5cars' # grab car info from official web
fh5_html = html = get_html(fh5_url, host='forza.net')
fh5_soup = BeautifulSoup(fh5_html)
all_car_info = fh5_soup.find_all('tbody')[0].find_all('tr')
all_car_info[:2]

[<tr>
 <td>2017 Abarth 124 Spider</td>
 <td>Modern Sports Cars</td>
 <td>Seasonal</td>
 <td>Series 24</td>
 </tr>,
 <tr>
 <td>2016 Abarth 695 Biposto</td>
 <td>Hot Hatch</td>
 <td>Seasonal</td>
 <td>Series 24</td>
 </tr>]

In [13]:
# format and combine all info here from official website
car_model_lst, car_type_lst, collect_lst, added_lst = [], [], [], [] 
for car_info in all_car_info:
    car_model, car_type, collect, added = [each.get_text() for each in car_info.find_all('td')]
    car_model_lst.append(car_model.upper())
    car_type_lst.append(car_type.upper())
    collect_lst.append(collect.upper())
    added_lst.append(added.upper())

df = pd.DataFrame({'CAR MODEL(Full Name)':car_model_lst, 
                   'CAR TYPE':car_type_lst, 
                   'COLLECT':collect_lst, 
                   'ADDED':added_lst})
df.head()

,CAR MODEL(Full Name),CAR TYPE,COLLECT,ADDED
0,2017 ABARTH 124 SPIDER,MODERN SPORTS CARS,SEASONAL,SERIES 24
1,2016 ABARTH 695 BIPOSTO,HOT HATCH,SEASONAL,SERIES 24
2,1980 ABARTH FIAT 131,CLASSIC RALLY,"BACKSTAGE, SEASONAL",SERIES 24
3,1968 ABARTH 595 ESSEESSE,CULT CARS,AUTOSHOW,SERIES 24
4,2017 ACURA NSX,MODERN SUPERCARS,AUTOSHOW,


In [14]:
len(df)

892

In [15]:
# find same car but different names in two above websites
car_model1 = df_kudo['CAR MODEL(Full Name)'].values
car_model2 = df['CAR MODEL(Full Name)'].values

miss_matched1 = []
for each in car_model1:
    if each not in car_model2:
        miss_matched1.append(each)

miss_matched2 = []
for each in car_model2:
    if each not in car_model1:
        miss_matched2.append(each)

In [ ]:
seen = []
for each in miss_matched1:
    corrected = find_closest_match(each, miss_matched2)
    if corrected in seen:
        print(each, corrected)
    else:
        seen.append(corrected)

In [17]:
len(seen), len(miss_matched1), len(miss_matched2) # find all matched names

(64, 64, 64)

In [18]:
# keep all names with official website
for i in range(len(miss_matched1)):
    df_kudo.loc[df_kudo['CAR MODEL(Full Name)']==miss_matched1[i], 'CAR MODEL(Full Name)'] = seen[i]

In [19]:
# integrate two dataframe
merged_df = pd.merge(df_kudo, df, on="CAR MODEL(Full Name)", how="inner")
merged_df.head()


,CAR MAKE,CAR MODEL(Full Name),CAR MODEL(Short Name),CAR TYPE,COLLECT,ADDED
0,ABARTH,1968 ABARTH 595 ESSEESSE,ABARTH 595 '68,CULT CARS,AUTOSHOW,SERIES 24
1,ABARTH,1980 ABARTH FIAT 131,ABARTH 131,CLASSIC RALLY,"BACKSTAGE, SEASONAL",SERIES 24
2,ABARTH,2016 ABARTH 695 BIPOSTO,ABARTH 695 '16,HOT HATCH,SEASONAL,SERIES 24
3,ABARTH,2017 ABARTH 124 SPIDER,ABARTH 124 '17,MODERN SPORTS CARS,SEASONAL,SERIES 24
4,ACURA,2001 ACURA INTEGRA TYPE-R,ACURA INTEGRA,RETRO HOT HATCH,AUTOSHOW,


In [20]:
merged_df.shape, len(merged_df['CAR MODEL(Full Name)'].unique()) # all matched and no duplicates

((892, 6), 892)

In [21]:
# start to work for script use. add columns
merged_df['CAR MAKE LOCATION'] = None
merged_df['CAR MODEL LOCATION'] = 0
merged_df['BUYOUT NUM'] = 0
merged_df = merged_df[['CAR MAKE', 'CAR MAKE LOCATION', 'CAR MODEL(Full Name)', 'CAR MODEL(Short Name)', 'CAR MODEL LOCATION', 'CAR TYPE','COLLECT', 'ADDED','BUYOUT NUM']]
merged_df.head()

,CAR MAKE,CAR MAKE LOCATION,CAR MODEL(Full Name),CAR MODEL(Short Name),CAR MODEL LOCATION,CAR TYPE,COLLECT,ADDED,BUYOUT NUM
0,ABARTH,None,1968 ABARTH 595 ESSEESSE,ABARTH 595 '68,0,CULT CARS,AUTOSHOW,SERIES 24,0
1,ABARTH,None,1980 ABARTH FIAT 131,ABARTH 131,0,CLASSIC RALLY,"BACKSTAGE, SEASONAL",SERIES 24,0
2,ABARTH,None,2016 ABARTH 695 BIPOSTO,ABARTH 695 '16,0,HOT HATCH,SEASONAL,SERIES 24,0
3,ABARTH,None,2017 ABARTH 124 SPIDER,ABARTH 124 '17,0,MODERN SPORTS CARS,SEASONAL,SERIES 24,0
4,ACURA,None,2001 ACURA INTEGRA TYPE-R,ACURA INTEGRA,0,RETRO HOT HATCH,AUTOSHOW,,0


In [22]:
sorted_make_list = sorted(make_list) # make sure it is sorted.
make_location_dict = dict(zip(sorted_make_list, [tuple() for _ in sorted_make_list]))
for i in range(len(sorted_make_list)):
    make_location_x = (i+1)%5
    make_location_y = (i+1)//5
    make_location_dict[sorted_make_list[i]] = (make_location_x, make_location_y)

In [23]:
# test
_name = 'PAGANI'
_name, make_location_dict[_name]

('PAGANI', (4, 19))

In [24]:
merged_df['CAR MAKE LOCATION'] = merged_df['CAR MAKE'].apply(lambda x: make_location_dict[x])
merged_df.head()

,CAR MAKE,CAR MAKE LOCATION,CAR MODEL(Full Name),CAR MODEL(Short Name),CAR MODEL LOCATION,CAR TYPE,COLLECT,ADDED,BUYOUT NUM
0,ABARTH,"(1, 0)",1968 ABARTH 595 ESSEESSE,ABARTH 595 '68,0,CULT CARS,AUTOSHOW,SERIES 24,0
1,ABARTH,"(1, 0)",1980 ABARTH FIAT 131,ABARTH 131,0,CLASSIC RALLY,"BACKSTAGE, SEASONAL",SERIES 24,0
2,ABARTH,"(1, 0)",2016 ABARTH 695 BIPOSTO,ABARTH 695 '16,0,HOT HATCH,SEASONAL,SERIES 24,0
3,ABARTH,"(1, 0)",2017 ABARTH 124 SPIDER,ABARTH 124 '17,0,MODERN SPORTS CARS,SEASONAL,SERIES 24,0
4,ACURA,"(2, 0)",2001 ACURA INTEGRA TYPE-R,ACURA INTEGRA,0,RETRO HOT HATCH,AUTOSHOW,,0


In [31]:
merged_df.to_csv('./FH5_all_cars_info_v1.csv', index=False) # store raw dataframe

In [18]:
merged_df

,CAR MAKE,CAR MAKE LOCATION,CAR MODEL(Full Name),CAR MODEL(Short Name),CAR MODEL LOCATION,CAR TYPE,COLLECT,ADDED,BUYOUT NUM
0,ABARTH,"(1, 0)",1968 ABARTH 595 ESSEESSE,ABARTH 595 '68,0,CULT CARS,AUTOSHOW,SERIES 24,0
1,ABARTH,"(1, 0)",1980 ABARTH FIAT 131,ABARTH 131,0,CLASSIC RALLY,"BACKSTAGE, SEASONAL",SERIES 24,0
2,ABARTH,"(1, 0)",2016 ABARTH 695 BIPOSTO,ABARTH 695 '16,0,HOT HATCH,SEASONAL,SERIES 24,0
3,ABARTH,"(1, 0)",2017 ABARTH 124 SPIDER,ABARTH 124 '17,0,MODERN SPORTS CARS,SEASONAL,SERIES 24,0
4,ACURA,"(2, 0)",2001 ACURA INTEGRA TYPE-R,ACURA INTEGRA,0,RETRO HOT HATCH,AUTOSHOW,NaN,0
...,...,...,...,...,...,...,...,...,...
887,WULING,"(2, 26)",2013 WULING SUNSHINE S,WULING SUNSHINE,0,VANS & UTILITY,SEASONAL,SERIES 4,0
888,WULING,"(2, 26)",2022 WULING HONGGUANG MINI EV MACARON,WULING MINI EV,0,HOT HATCH,AUTOSHOW DLC,CHINESE LUCKY STARS CAR PACK,0
889,XPENG,"(3, 26)",2020 XPENG P7,XPENG P7,0,SUPER SALOONS,AUTOSHOW,SERIES 11,0
890,ZENVO,"(4, 26)",2016 ZENVO ST1,ZENVO ST1,0,HYPERCARS,SEASONAL,SERIES 3,0


In [17]:
grab_info_df = pd.read_csv('./OCR_model_short_name.csv')
grab_info_df.head()

,CAR MODEL(Short Name),ORDER
0,ABARTH 124 '17,1
1,ABARTH 131,2
2,ABARTH 595 '68,3
3,ABARTH 695 '16,4
4,ACURA INTEGRA,1


In [43]:
duplicated_short_name = grab_info_df[grab_info_df['CAR MODEL(Short Name)'].duplicated()]['CAR MODEL(Short Name)'].values
grab_info_df.loc[grab_info_df['CAR MODEL(Short Name)'].isin(duplicated_short_name)]

,CAR MODEL(Short Name),ORDER
29,AM DBS,2
31,AM DBS,4
389,HONDA 52000,14
390,HONDA 52000,15
749,PORSCHE CAYMAN,43
750,PORSCHE CAYMAN,44


In [45]:
# note Honda and porsche has duplicated bugs
grab_info_df.loc[29, 'CAR MODEL(Short Name)'] = 'AM DB5'
grab_info_df.loc[389, 'CAR MODEL(Short Name)'] = 'HONDA S2000'
grab_info_df.loc[390, 'CAR MODEL(Short Name)'] = 'HONDA S2000'

In [46]:
cnt = 0
for each in grab_info_df['CAR MODEL(Short Name)']:
    if each in merged_df['CAR MODEL(Short Name)'].values:
        cnt += 1

print(f"number of matched short name: {cnt}")

number of matched short name: 512


In [47]:
df_all = pd.merge(merged_df, grab_info_df, on="CAR MODEL(Short Name)", how='left')
df_all.shape

(895, 10)

In [59]:
df_all['CAR MODEL LOCATION'] = df_all['ORDER']
df_all=df_all.drop(columns=['ORDER'])
df_all.head()

,CAR MAKE,CAR MAKE LOCATION,CAR MODEL(Full Name),CAR MODEL(Short Name),CAR MODEL LOCATION,CAR TYPE,COLLECT,ADDED,BUYOUT NUM
0,ABARTH,"(1, 0)",1968 ABARTH 595 ESSEESSE,ABARTH 595 '68,3.0,CULT CARS,AUTOSHOW,SERIES 24,0
1,ABARTH,"(1, 0)",1980 ABARTH FIAT 131,ABARTH 131,2.0,CLASSIC RALLY,"BACKSTAGE, SEASONAL",SERIES 24,0
2,ABARTH,"(1, 0)",2016 ABARTH 695 BIPOSTO,ABARTH 695 '16,4.0,HOT HATCH,SEASONAL,SERIES 24,0
3,ABARTH,"(1, 0)",2017 ABARTH 124 SPIDER,ABARTH 124 '17,1.0,MODERN SPORTS CARS,SEASONAL,SERIES 24,0
4,ACURA,"(2, 0)",2001 ACURA INTEGRA TYPE-R,ACURA INTEGRA,1.0,RETRO HOT HATCH,AUTOSHOW,NaN,0


In [60]:
df_all = df_all.to_csv('./FH5_all_cars_info_v2.csv', index=False) # store 2ndraw dataframe

In [1]:
import pandas as pd

# Reading an Excel file
# Ensure the file exists in the directory or provide the correct path
file_path = "FH5_all_cars_info_v3.xlsx"
data = pd.read_excel(file_path)  # Reads the first sheet by default

# Writing to a new Excel file
output_path = "./FH5_all_cars_info_v4.xlsx"
data.to_excel(output_path, index=False)  # Avoid writing the index column

In [2]:
data.head()

,CAR MAKE,CAR MAKE LOCATION,CAR MODEL(Full Name),CAR MODEL(Short Name),CAR MODEL LOCATION,CAR TYPE,COLLECT,ADDED,BUYOUT NUM
0,ABARTH,"(1, 0)",1968 ABARTH 595 ESSEESSE,ABARTH 595 '68,3,CULT CARS,AUTOSHOW,SERIES 24,0
1,ABARTH,"(1, 0)",1980 ABARTH FIAT 131,ABARTH 131,2,CLASSIC RALLY,"BACKSTAGE, SEASONAL",SERIES 24,0
2,ABARTH,"(1, 0)",2016 ABARTH 695 BIPOSTO,ABARTH 695 '16,4,HOT HATCH,SEASONAL,SERIES 24,0
3,ABARTH,"(1, 0)",2017 ABARTH 124 SPIDER,ABARTH 124 '17,1,MODERN SPORTS CARS,SEASONAL,SERIES 24,0
4,ACURA,"(2, 0)",2001 ACURA INTEGRA TYPE-R,ACURA INTEGRA,1,RETRO HOT HATCH,AUTOSHOW,NaN,0


In [5]:
import pandas as pd
from openpyxl import load_workbook


file_path = "./FH5_all_cars_info_v3.xlsx"

# Adjust column widths using openpyxl
workbook = load_workbook(file_path)
sheet = workbook.active

sheet.auto_filter.ref = sheet.dimensions

for col in sheet.columns:
    max_length = 0
    col_letter = col[0].column_letter
    for cell in col:
        try:
            # Calculate the maximum length of values in the column (including the header)
            max_length = max(max_length, len(str(cell.value)))
        except:
            pass
    # Adjust the column width
    sheet.column_dimensions[col_letter].width = max_length + 2  # Add some padding

# Save the workbook with adjusted widths
workbook.save("./FH5_all_cars_info_v3.xlsx")
print(f"Adjusted column widths saved to {'./FH5_all_cars_info_v3.xlsx'}")


Adjusted column widths saved to ./FH5_all_cars_info_v3.xlsx


In [10]:
import pandas as pd
from openpyxl import load_workbook

file_path = "./FH5_all_cars_info_v3.xlsx"
data = pd.read_excel(file_path)  # Reads the first sheet by default

sheet_name = "all_cars_info"

# Write the DataFrame to an Excel file with the specified sheet name
output_path = "./FH5_all_cars_info_v3.xlsx"
data.to_excel(output_path, index=False, sheet_name=sheet_name)

workbook = load_workbook(file_path)
sheet = workbook.active

sheet.auto_filter.ref = sheet.dimensions

for col in sheet.columns:
    max_length = 0
    col_letter = col[0].column_letter
    for cell in col:
        try:
            # Calculate the maximum length of values in the column (including the header)
            max_length = max(max_length, len(str(cell.value)))
        except:
            pass
    # Adjust the column width
    sheet.column_dimensions[col_letter].width = max_length + 2  # Add some padding

# Save the workbook with adjusted widths
workbook.save("./FH5_all_cars_info_v3.xlsx")